In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chama

In [3]:
#Define search area boundary as dict()
boundary = {"min x":0,
            "min y": 0,
            "max x": 12000,
            "max y": 30,
            "min z":0,
            "max z": 12000
            }

grid_distance = 10

node_x = int((boundary["max x"] - boundary["min x"])/grid_distance)+1
node_y = int((boundary["max y"] - boundary["min y"])/grid_distance)+1
node_z = 21
# Create the Simulation gridspace
# Create the Simulation gridspace
x_grid = np.linspace(0, boundary["max x"], node_x)
y_grid = np.linspace(boundary["min y"], boundary["max y"], node_y)
y_grid = np.array([10]) # 10m to the side
z_grid = np.linspace(boundary["min z"], boundary["max z"], node_z)
z_grid = np.linspace(0,boundary["max z"],node_z)
#np.array([1, 2, 3, 5, 10, 15])
grid = chama.simulation.Grid(x_grid, y_grid, z_grid)

In [4]:
# Values 
wind_speeds = [3,10]
Stability_classes = ['A','B','C','D','E','F']
heights = [10]
source_rates = np.array([25.0,50.0,100.0])/3600 #rates in Kg/hr

In [5]:
#Low Wind (3m/s)
# all release rates
# both heights
scenario_num = 1
Source_position_1 = [0, 10, heights[0], source_rates[0]]
source_1 = chama.simulation.Source(Source_position_1[0], Source_position_1[1], Source_position_1[2], Source_position_1[3])

atm = pd.DataFrame({'Wind Direction': [0],
            'Wind Speed': [wind_speeds[0]],
              'Stability Class': [Stability_classes[0]]}, index=[0])
gauss_plume = chama.simulation.GaussianPlume(grid, source_1, atm, density_eff = 0.664)
signalGuassPlume = gauss_plume.conc
column_name = f'S {scenario_num}'
print(signalGuassPlume.head())
signalGuassPlume = signalGuassPlume.rename(columns={'S':column_name})
print(signalGuassPlume.head())
signalGuassPlume.insert(4, 'S_Class', Stability_classes[0])
signalGuassPlume.insert(5, 'W_Speed', wind_speeds[0])
signalGuassPlume.insert(6, 'R_rate', source_rates[0])
signalGuassPlume.insert(7, 'H', heights[0])
print(signalGuassPlume.head())
signal = signalGuassPlume


     X     Y     Z  T    S
0  0.0  10.0   0.0  0  0.0
1  0.0  10.0  10.0  0  0.0
2  0.0  10.0  20.0  0  0.0
3  0.0  10.0  30.0  0  0.0
4  0.0  10.0  40.0  0  0.0
     X     Y     Z  T  S 1
0  0.0  10.0   0.0  0  0.0
1  0.0  10.0  10.0  0  0.0
2  0.0  10.0  20.0  0  0.0
3  0.0  10.0  30.0  0  0.0
4  0.0  10.0  40.0  0  0.0
     X     Y     Z  T S_Class  W_Speed    R_rate   H  S 1
0  0.0  10.0   0.0  0       A        3  0.006944  10  0.0
1  0.0  10.0  10.0  0       A        3  0.006944  10  0.0
2  0.0  10.0  20.0  0       A        3  0.006944  10  0.0
3  0.0  10.0  30.0  0       A        3  0.006944  10  0.0
4  0.0  10.0  40.0  0       A        3  0.006944  10  0.0


In [6]:
signal = signal.rename(columns={'S 1':column_name})

In [8]:
s_class = 'A'
# Values 
wind_speeds = [3,10]
Stability_classes = ['A','D','F']
heights = [1, 10]
source_rates = np.array([25.0, 50.0])/3600 #

conversion = 1416000
scenario_num = 1

for source_rate in source_rates:
    for s_class in Stability_classes:
        for Sensor_h  in [10, 50, 200, 400, 500,600, 800]: #sensor heights
            fig = plt.figure()
            i = 111
            ax = plt.subplot(111)
            for height in heights:
                for wind_speed in wind_speeds:
                    Source_position_1 = [0, 10, height, source_rate]
                    source_1 = chama.simulation.Source(Source_position_1[0], Source_position_1[1], Source_position_1[2], Source_position_1[3])

                    atm = pd.DataFrame({'Wind Direction': [0],
                                'Wind Speed': [wind_speed],
                                  'Stability Class': [s_class]}, index=[0])
                    gauss_plume = chama.simulation.GaussianPlume(grid, source_1, atm, density_eff = 0.664)
                    signalGuassPlume = gauss_plume.conc
                    column_name = f'U {wind_speed} H {height}'
                    signalGuassPlume = signalGuassPlume.rename(columns={'S':column_name})
                    signalGuassPlume.insert(4, 'S_Class', s_class)
                    signalGuassPlume.insert(5, 'W_Speed', wind_speed)
                    signalGuassPlume.insert(6, 'R_rate', source_rate)
                    signalGuassPlume.insert(7, 'H', heights[0])
                    signal = signalGuassPlume.loc[signalGuassPlume['Z'] == Sensor_h]
                    signal = signal[column_name]*conversion
                    title = f'Sensor Height: {Sensor_h} (m), S Class: {s_class}, Release rate: {source_rate*3600} (kg/hr)'
                    logg = True
                    if wind_speed == 3: L_color = 'r'
                    else: L_color = 'b'
                    y_lim = (0.0001,1000)
                    if height == 1:
                        ax = signal.plot(x='X',
                                    y= f'H {height}',
                                    legend=True,
                                    title=title,
                                    ax = ax,
                                    xlabel='Sensor Position Downwind (m)',
                                    ylabel='Concentration (ppm)',
                                    logy = logg,
                                    color = L_color,
                                    ylim = y_lim,
                                    xlim = (10,1000),
                                    linestyle = 'dotted')
                    elif height == 5:
                        ax = signal.plot(x='X',
                                    y= f'H {height}',
                                    legend=True,
                                    title=title,
                                    ax = ax,
                                    xlabel='Sensor Position Downwind (m)',
                                    ylabel='Concentration (ppm)',
                                    logy = logg,
                                    color = L_color,
                                    ylim = y_lim,
                                    xlim = (10,1000),
                                    linestyle = 'dashed')
                    elif height == 5:
                                    ax = signal.plot(x='X',
                                    y= f'H {height}',
                                    legend=True,
                                    title=title,
                                    ax = ax,
                                    xlabel='Sensor Position Downwind (m)',
                                    ylabel='Concentration (ppm)',
                                    logy = logg,
                                    color = L_color,
                                    ylim = y_lim,
                                    xlim = (10,1000),
                                    linestyle = 'dashed')
                    else:
                        ax = signal.plot(x='X',
                                    y= f'H {height}',
                                    legend=True,
                                    title=title,
                                    ax = ax,
                                    xlabel='Sensor Position Downwind (m)',
                                    ylabel='Concentration (ppm)',
                                    ylim = y_lim,
                                    xlim = (10,1000),
                                    logy = logg)
            x = np.arange(10,200,10)
            a = np.ones(len(x))
            y1 = a*50
            y2 = a*20
            y3 = a*1

            ax.plot(x, y1, alpha=0.7,label='NN 50 ppm')
            ax.plot(x, y2, alpha=0.7,label='SGX 20 ppm')
            ax.plot(x, y3, alpha=0.7,label='TDLAS  1 ppm')
            
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

            # Put a legend to the right of the current axis
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

            fname = f'Results/Log Concentration plot S Class {s_class} Sensor_h {Sensor_h} R {source_rate*3600}.png'
            plt.savefig(fname)
            plt.clf()
            plt.close()


In [14]:
signalGuassPlume

,X,Y,Z,T,S_Class,W_Speed,R_rate,H,U 10 H 10
0,0.0,10.0,0.0,0,F,10,0.013889,1,0.0
1,0.0,10.0,10.0,0,F,10,0.013889,1,0.0
2,0.0,10.0,20.0,0,F,10,0.013889,1,0.0
3,0.0,10.0,30.0,0,F,10,0.013889,1,0.0
4,0.0,10.0,40.0,0,F,10,0.013889,1,0.0
...,...,...,...,...,...,...,...,...,...
1001996,10000.0,10.0,9960.0,0,F,10,0.013889,1,0.0
1001997,10000.0,10.0,9970.0,0,F,10,0.013889,1,0.0
1001998,10000.0,10.0,9980.0,0,F,10,0.013889,1,0.0
1001999,10000.0,10.0,9990.0,0,F,10,0.013889,1,0.0


In [35]:
def get_conc_at_point(source_rate, s_class, Sensor_h, height,wind_speed):
    boundary = {"min x":0,
            "min y": 0,
            "max x": 10000,
            "max y": 30,
            "min z": 0,
            "max z": 10000
            }

    grid_distance = 50

    node_x = int((boundary["max x"] - boundary["min x"])/grid_distance)+1
    node_y = int((boundary["max y"] - boundary["min y"])/grid_distance)+1
    node_z = int((boundary["max z"] - boundary["min z"])/grid_distance)+1
    
    # Create the Simulation gridspace
    x_grid = np.linspace(0, boundary["max x"], node_x)
    y_grid = np.linspace(boundary["min y"], boundary["max y"], node_y)
    y_grid = np.array([10]) # 10m to the side
    z_grid = np.linspace(boundary["min z"], boundary["max z"], node_z)
    z_grid = np.linspace(0,boundary["max z"],node_z)
    
    #np.array([1, 2, 3, 5, 10, 15])
    grid = chama.simulation.Grid(x_grid, y_grid, z_grid)
    Source_position_1 = [0, 10, height, source_rate]
    source_1 = chama.simulation.Source(Source_position_1[0], Source_position_1[1], Source_position_1[2], Source_position_1[3])

    atm = pd.DataFrame({'Wind Direction': [0],
                'Wind Speed': [wind_speed],
                  'Stability Class': [s_class]}, index=[0])
    gauss_plume = chama.simulation.GaussianPlume(grid, source_1, atm, density_eff = 0.664)
    signalGuassPlume = gauss_plume.conc
    column_name = f'U {wind_speed} H {height}'
    signalGuassPlume = signalGuassPlume.rename(columns={'S':column_name})
    signalGuassPlume.insert(4, 'S_Class', s_class)
    signalGuassPlume.insert(5, 'W_Speed', wind_speed)
    signalGuassPlume.insert(6, 'R_rate', source_rate)
    signalGuassPlume.insert(7, 'H', heights[0])
    signal = signalGuassPlume.loc[signalGuassPlume['Z'] == Sensor_h]
    signal = signal[column_name]*conversion
    return signal, signalGuassPlume

In [54]:
X, Plume = get_conc_at_point(source_rate= 50, s_class='A', Sensor_h='20', height=10, wind_speed=5)

In [17]:
signal = signalGuassPlume.loc[signalGuassPlume['Z'] == Sensor_h]

Series([], Name: U 5 H 10, dtype: float64)

In [57]:
P_2 = Plume.loc[Plume['U 5 H 10'] > 0.00001]

In [58]:
P_3 = P_2.loc[P_2['Z'] > 500]
print(P_3)

           X     Y      Z  T S_Class  W_Speed  R_rate  H  U 5 H 10
3227   800.0  10.0  550.0  0       A        5      50  1  0.000012
3428   850.0  10.0  550.0  0       A        5      50  1  0.000013
3429   850.0  10.0  600.0  0       A        5      50  1  0.000010
3629   900.0  10.0  550.0  0       A        5      50  1  0.000015
3630   900.0  10.0  600.0  0       A        5      50  1  0.000012
3830   950.0  10.0  550.0  0       A        5      50  1  0.000015
3831   950.0  10.0  600.0  0       A        5      50  1  0.000013
3832   950.0  10.0  650.0  0       A        5      50  1  0.000011
4031  1000.0  10.0  550.0  0       A        5      50  1  0.000015
4032  1000.0  10.0  600.0  0       A        5      50  1  0.000013
4033  1000.0  10.0  650.0  0       A        5      50  1  0.000011
4232  1050.0  10.0  550.0  0       A        5      50  1  0.000015
4233  1050.0  10.0  600.0  0       A        5      50  1  0.000014
4234  1050.0  10.0  650.0  0       A        5      50  1  0.00